In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', 26)

In [5]:
# upload dataset to colab
from google.colab import files
uploaded = files.upload()

Saving ML_Features_1144.csv to ML_Features_1144 (1).csv


In [27]:
# read dataset to pandas dataframe
import io

df = pd.read_csv(io.BytesIO(uploaded['ML_Features_1144.csv']))
df.head()

,Unnamed: 0,image.name,X.x,Blue.count,red.count,Blue.percentage,red.percentage,total.clusters,average.clusters.32,area,circularity,X.1,X.y,...,Texture_SumEntropy_3_0,Texture_SumEntropy_3_135,Texture_SumEntropy_3_45,Texture_SumEntropy_3_90,Texture_SumVariance_3_0,Texture_SumVariance_3_135,Texture_SumVariance_3_45,Texture_SumVariance_3_90,Texture_Variance_3_0,Texture_Variance_3_135,Texture_Variance_3_45,Texture_Variance_3_90,classification
0,1,Case 3 A10-10547-25283,548,16611,52475,1.584148,5.004406,123,0.112844,81.252033,0.081106,1,1,...,1.251311,1.272596,1.279650,1.269986,4.985359,4.654160,4.499266,4.718412,1.545288,1.538943,1.542346,1.541256,Non-Tumor
1,2,Case 3 A10-10566-40206,549,93148,282307,8.883286,26.922894,143,0.131193,69.671329,0.119117,2,2,...,1.657042,1.708659,1.697252,1.689416,24.047674,23.805470,23.886660,23.949666,6.115417,6.105633,6.105821,6.109401,Non-Tumor
2,3,Case 3 A10-13444-20223,550,107853,198888,10.285664,18.967438,166,0.152294,75.361446,0.094944,3,3,...,1.652502,1.678369,1.701316,1.668477,23.754724,23.636317,23.338890,23.673391,6.170840,6.175347,6.176097,6.171856,Non-Tumor
3,4,Case 3 A10-14507-37285,551,58609,208594,5.589390,19.893074,153,0.140367,68.398693,0.134178,4,4,...,1.778280,1.839086,1.850145,1.839611,20.562092,20.061859,19.908849,20.037982,5.341032,5.336172,5.342228,5.338274,Non-Tumor
4,5,Case 3 A10-14726-26052,552,13419,56428,1.279736,5.381393,81,0.074312,82.679012,0.096307,5,5,...,1.171170,1.199718,1.204215,1.197788,3.977051,3.552465,3.481328,3.593105,1.252694,1.253966,1.253085,1.253502,Non-Tumor


In [28]:
# Remove uneeded columns & perform feature selection
df.drop('Unnamed: 0', axis = 1, inplace = True)
df.drop('image.name', axis = 1, inplace = True)
df.drop('X.x', axis = 1, inplace = True)
df.drop('X.1', axis = 1, inplace = True)
df.drop('X.y', axis = 1, inplace = True)
df.drop('Count_Nuclei', axis = 1, inplace = True)
df.drop('ImageNumber', axis = 1, inplace = True)
df.drop('total.clusters', axis = 1, inplace = True)
df.drop('average.clusters.32', axis = 1, inplace = True)

texture_columns = [col for col in df.columns if "Texture" in col]
for col in texture_columns:
  df.drop(col, axis = 1, inplace = True)

In [8]:
df.head()

,Blue.count,red.count,Blue.percentage,red.percentage,area,circularity,classification
0,16611,52475,1.584148,5.004406,81.252033,0.081106,Non-Tumor
1,93148,282307,8.883286,26.922894,69.671329,0.119117,Non-Tumor
2,107853,198888,10.285664,18.967438,75.361446,0.094944,Non-Tumor
3,58609,208594,5.589390,19.893074,68.398693,0.134178,Non-Tumor
4,13419,56428,1.279736,5.381393,82.679012,0.096307,Non-Tumor


In [9]:
# View data info
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1144 entries, 0 to 1143
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Blue.count       1144 non-null   int64  
 1   red.count        1144 non-null   int64  
 2   Blue.percentage  1144 non-null   float64
 3   red.percentage   1144 non-null   float64
 4   area             1143 non-null   float64
 5   circularity      1143 non-null   float64
 6   classification   1144 non-null   object 
dtypes: float64(4), int64(2), object(1)
memory usage: 62.7+ KB


In [29]:
# Separate numeric and categorical (object) data
num_columns = [col for col in df.columns if df[col].dtype != 'object']
cat_columns = [col for col in df.columns if df[col].dtype == 'object']

In [11]:
num_columns

['Blue.count',
 'red.count',
 'Blue.percentage',
 'red.percentage',
 'area',
 'circularity']

In [12]:
cat_columns

['classification']

In [13]:
# Check available values of each category column
for col in cat_columns:
    print(f"{col} has {df[col].unique()} values\n")

classification has ['Non-Tumor' 'Non-Viable-Tumor' 'Viable' 'viable: non-viable'] values



In [30]:
# Check for null values
df.isna().sum()

Blue.count         0
red.count          0
Blue.percentage    0
red.percentage     0
area               1
circularity        1
classification     0
dtype: int64

In [31]:
# mean/mode sampling
def impute_method(feature):
    frequent_feature = df[feature].mode()[0]
    df[feature] = df[feature].fillna(frequent_feature)

In [32]:
# Fill numeric columns using mean/mode sampling
for col in num_columns:
    impute_method(col)

In [17]:
df.isna().sum()

Blue.count         0
red.count          0
Blue.percentage    0
red.percentage     0
area               0
circularity        0
classification     0
dtype: int64

In [34]:
# Convert categorical data into labels of 1 and 0
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
for col in cat_columns:
  df[col] = encoder.fit_transform(df[col])

In [19]:
df.head()

,Blue.count,red.count,Blue.percentage,red.percentage,area,circularity,classification
0,16611,52475,1.584148,5.004406,81.252033,0.081106,0
1,93148,282307,8.883286,26.922894,69.671329,0.119117,0
2,107853,198888,10.285664,18.967438,75.361446,0.094944,0
3,58609,208594,5.589390,19.893074,68.398693,0.134178,0
4,13419,56428,1.279736,5.381393,82.679012,0.096307,0


In [20]:
for col in cat_columns:
    print(f"{col} has {df[col].unique()} values\n")

classification has [0 1 2 3] values



In [21]:
# Split data
X = df.drop(columns = 'classification', axis = 1)
Y = df['classification']

In [22]:
# Split data into train and test
from sklearn.model_selection import train_test_split

x_train , x_test , y_train , y_test = train_test_split(X, Y, test_size = .2, random_state = 42)

In [23]:
# Create multiple models to obtain best model

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

models = []
models.append(('LogisticRegression', LogisticRegression(solver='liblinear')))
models.append(('Naive bayes', GaussianNB()))
models.append(('Random Forest', RandomForestClassifier()))
models.append(('Decision Tree', DecisionTreeClassifier()))
models.append(('KNN', KNeighborsClassifier()))

In [24]:
for name, model in models:
    model.fit(x_train, y_train)
    prediction = model.predict(x_test)
    print(name)
    print(confusion_matrix(prediction, y_test))
    print(accuracy_score(prediction,y_test))
    print('\n')

LogisticRegression
[[98 47 35  5]
 [ 0  0  0  0]
 [ 9  7 24  4]
 [ 0  0  0  0]]
0.5327510917030568


Naive bayes
[[67 18  6  0]
 [23 29 12  0]
 [17  7 41  9]
 [ 0  0  0  0]]
0.5982532751091703


Random Forest
[[85 21 13  3]
 [13 28  7  1]
 [ 8  4 38  5]
 [ 1  1  1  0]]
0.6593886462882096


Decision Tree
[[63 20 16  4]
 [22 31  7  3]
 [17  2 32  1]
 [ 5  1  4  1]]
0.5545851528384279


KNN
[[86 27 17  2]
 [14 22 16  3]
 [ 6  5 25  3]
 [ 1  0  1  1]]
0.5851528384279476




In [33]:
# Accuracy seems bad, therefore we'll combine categories a create only a binary classification
df['classification'].replace(to_replace = "Non-Viable-Tumor" , value ="Non-Tumor", inplace = True)
df['classification'].replace(to_replace = "viable: non-viable" , value ="Viable", inplace = True)

In [35]:
for col in cat_columns:
    print(f"{col} has {df[col].unique()} values\n")

classification has [0 1] values



In [36]:
X = df.drop(columns = 'classification', axis = 1)
Y = df['classification']

In [37]:
from sklearn.model_selection import train_test_split

x_train , x_test , y_train , y_test = train_test_split(X, Y, test_size = .2, random_state = 42)

In [38]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

models = []
models.append(('LogisticRegression', LogisticRegression(solver='liblinear')))
models.append(('Naive bayes', GaussianNB()))
models.append(('Random Forest', RandomForestClassifier()))
models.append(('Decision Tree', DecisionTreeClassifier()))
models.append(('KNN', KNeighborsClassifier()))

In [39]:
for name, model in models:
    model.fit(x_train, y_train)
    prediction = model.predict(x_test)
    print(name)
    print(confusion_matrix(prediction, y_test))
    print(accuracy_score(prediction,y_test))
    print('\n')

LogisticRegression
[[155  57]
 [  6  11]]
0.7248908296943232


Naive bayes
[[133  17]
 [ 28  51]]
0.8034934497816594


Random Forest
[[148  30]
 [ 13  38]]
0.8122270742358079


Decision Tree
[[137  30]
 [ 24  38]]
0.7641921397379913


KNN
[[145  33]
 [ 16  35]]
0.7860262008733624




In [40]:
# Predict test set
model = models[2][1]
prediction = model.predict(x_test)   
print('Random Forest')
print(confusion_matrix(prediction, y_test))
print(accuracy_score(prediction, y_test))
print('\n')

Random Forest
[[148  30]
 [ 13  38]]
0.8122270742358079




In [41]:
# Save random forest model
import pickle
with open("Best-Model-Random-Forest.pickle", "wb") as f:
    pickle.dump(models[2][1], f)